In [1]:
pip install holidays

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.6/184.6 KB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 46.9 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pymeeus: filename=PyMeeus-0.5.11-py3-none-any.whl size=730984 sha256=4a02f6ac216271822a2152be539d4504a56a073f67af77565016b96a976fcef6
  Stored in directory: /home/ec2-user/.cache/pip/wheels/a0/8b/b2/810ae5a6f970c8be4725353400d643c90de1c0f023a9884ee7
Successfully built pymeeus
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow2_p38/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [20]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import holidays
import time

from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import tensorflow as tf


s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'Tabular-Playground-Series/Tabular-Playground-Sep-2022/train.csv'
file_key_2 = 'Tabular-Playground-Series/Tabular-Playground-Sep-2022/test.csv'
file_key_3 = 'Tabular-Playground-Series/Tabular-Playground-Sep-2022/sample_submission.csv'
file_key_4 = 'Tabular-Playground-Series/Tabular-Playground-Sep-2022/TPSSEP22_GDP_data_2017_to_2021.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

bucket_object_4 = bucket.Object(file_key_4)
file_object_4 = bucket_object_4.get()
file_content_stream_4 = file_object_4.get('Body')

## Reading data-files
train = pd.read_csv(file_content_stream_1)
train['date'] = pd.to_datetime(train['date'], format = '%Y-%m-%d')

test = pd.read_csv(file_content_stream_2)
test['date'] = pd.to_datetime(test['date'], format = '%Y-%m-%d')

submission = pd.read_csv(file_content_stream_3)
country_gdp = pd.read_csv(file_content_stream_4)

## Basic feature engineering 
train['weekday'] = train['date'].dt.dayofweek
train['month'] = train['date'].dt.month
train['weekend'] = np.where(train['weekday'] >= 5, 1, 0)
train['dayOfMonth'] = train['date'].dt.day
train['dayOfYear'] = train['date'].dt.dayofyear
train['quarter'] = train['date'].dt.quarter
train['year'] = train['date'].dt.year

test['weekday'] = test['date'].dt.dayofweek
test['month'] = test['date'].dt.month
test['weekend'] = np.where(test['weekday'] >= 5, 1, 0)
test['dayOfMonth'] = test['date'].dt.day
test['dayOfYear'] = test['date'].dt.dayofyear
test['quarter'] = test['date'].dt.quarter
test['year'] = test['date'].dt.year

## Appending GDP
train = pd.merge(train, country_gdp, on = ['country', 'year'], how = 'left')
train = train.drop(columns = ['year'], axis = 1)

test = pd.merge(test, country_gdp, on = ['country', 'year'], how = 'left')
test = test.drop(columns = ['year'], axis = 1)

## Extracting holidays
be_holidays = holidays.BE(years = [2017, 2018, 2019, 2020, 2021])
fr_holidays = holidays.FR(years = [2017, 2018, 2019, 2020, 2021])
de_holidays = holidays.DE(years = [2017, 2018, 2019, 2020, 2021])
it_holidays = holidays.IT(years = [2017, 2018, 2019, 2020, 2021])
pl_holidays = holidays.PL(years = [2017, 2018, 2019, 2020, 2021])
es_holidays = holidays.ES(years = [2017, 2018, 2019, 2020, 2021])

train_list = list()
test_list = list()
countries = ['Belgium', 'France', 'Germany', 'Italy', 'Poland', 'Spain']

for i in range(0, len(countries)):
    
    train_temp = train[train['country'] == countries[i]].reset_index(drop = True)
    train_temp['is_holiday'] = np.nan
    train_temp['holiday_season'] = np.nan
     
    test_temp = test[test['country'] == countries[i]].reset_index(drop = True)
    test_temp['is_holiday'] = np.nan
    test_temp['holiday_season'] = np.nan
    
    if (i == 0):
        
        holiday_to_use = be_holidays
        
    elif (i == 1):
        
        holiday_to_use = fr_holidays
        
    elif (i == 2):
        
        holiday_to_use = de_holidays
        
    elif (i == 3):
        
        holiday_to_use = it_holidays
        
    elif (i == 4):
        
        holiday_to_use = pl_holidays
        
    else:
        
        holiday_to_use = es_holidays
    
    for j in range(0, train_temp.shape[0]):
        
        train_temp['is_holiday'][j] = np.where(train_temp['date'][j] in holiday_to_use, 1, 0)
        
        if ((train_temp['date'][j] >= pd.to_datetime('2017-12-15')) & (train_temp['date'][j] <= pd.to_datetime('2017-12-31'))):
            
            train_temp['holiday_season'][j] = 1
            
        elif ((train_temp['date'][j] >= pd.to_datetime('2018-12-15')) & (train_temp['date'][j] <= pd.to_datetime('2018-12-31'))):    
            
            train_temp['holiday_season'][j] = 1
              
        elif ((train_temp['date'][j] >= pd.to_datetime('2019-12-15')) & (train_temp['date'][j] <= pd.to_datetime('2019-12-31'))):      
            
            train_temp['holiday_season'][j] = 1
       
        elif ((train_temp['date'][j] >= pd.to_datetime('2020-12-15')) & (train_temp['date'][j] <= pd.to_datetime('2020-12-31'))):
            
            train_temp['holiday_season'][j] = 1
        
        else:
            
            train_temp['holiday_season'][j] = 0
            
    train_list.append(train_temp)
    
    for k in range(0, test_temp.shape[0]):
        
        test_temp['is_holiday'][k] = np.where(test_temp['date'][k] in holiday_to_use, 1, 0)
        
        if ((test_temp['date'][k] >= pd.to_datetime('2017-12-15')) & (test_temp['date'][k] <= pd.to_datetime('2017-12-31'))):
            
            test_temp['holiday_season'][j] = 1
            
        elif ((test_temp['date'][k] >= pd.to_datetime('2018-12-15')) & (test_temp['date'][k] <= pd.to_datetime('2018-12-31'))):    
            
            test_temp['holiday_season'][j] = 1
              
        elif ((test_temp['date'][k] >= pd.to_datetime('2019-12-15')) & (test_temp['date'][k] <= pd.to_datetime('2019-12-31'))):      
            
            test_temp['holiday_season'][j] = 1
       
        elif ((test_temp['date'][k] >= pd.to_datetime('2020-12-15')) & (test_temp['date'][k] <= pd.to_datetime('2020-12-31'))):
            
            test_temp['holiday_season'][k] = 1
        
        else:
            
            test_temp['holiday_season'][k] = 0
            
    test_list.append(test_temp)
    
## Putting train and test in the right format
train = pd.concat(train_list)
train['is_holiday'] = train['is_holiday'].astype(int)
train['holiday_season'] = train['holiday_season'].astype(int)

test = pd.concat(test_list)
test['is_holiday'] = test['is_holiday'].astype(int)
test['holiday_season'] = test['holiday_season'].astype(int)

/tmp/ipykernel_10123/1660720441.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_temp['is_holiday'][j] = np.where(train_temp['date'][j] in holiday_to_use, 1, 0)
/tmp/ipykernel_10123/1660720441.py:143: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_temp['holiday_season'][j] = 0
/tmp/ipykernel_10123/1660720441.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_temp['holiday_season'][j] = 1
/tmp/ipykernel_10123/16

In [21]:
train.head()

,row_id,date,country,store,product,num_sold,weekday,month,weekend,dayOfMonth,dayOfYear,quarter,GDP,is_holiday,holiday_season
0,0,2017-01-01,Belgium,KaggleMart,Kaggle Advanced Techniques,663,6,1,1,1,1,1,0.5015,1,0
1,1,2017-01-01,Belgium,KaggleMart,Kaggle Getting Started,615,6,1,1,1,1,1,0.5015,1,0
2,2,2017-01-01,Belgium,KaggleMart,Kaggle Recipe Book,480,6,1,1,1,1,1,0.5015,1,0
3,3,2017-01-01,Belgium,KaggleMart,Kaggle for Kids: One Smart Goose,710,6,1,1,1,1,1,0.5015,1,0
4,4,2017-01-01,Belgium,KaggleRama,Kaggle Advanced Techniques,240,6,1,1,1,1,1,0.5015,1,0


In [22]:
all_data = pd.concat([train, test], axis = 0)

le = LabelEncoder()
cols = ['country', 'store', 'product']
for col in cols:
    le = LabelEncoder()
    all_data[col] = le.fit_transform(all_data[col])

scaler = MinMaxScaler()    

all_data = all_data.drop(['date', 'row_id'], axis = 1)
train = all_data.iloc[:70128,:]
test = all_data.iloc[70128:,:].drop(['num_sold'], axis = 1)
test = pd.DataFrame(scaler.fit_transform(test), columns = test.columns)

X = train.drop(['num_sold'], axis = 1)
Y = np.log(train['num_sold'])

t1 = time.time()
kf = KFold(n_splits = 4, shuffle = True, random_state = 888)
score_list_tf = []
test_preds_tf = []
fold = 1

## Defining model 
model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(16, input_dim = 12, activation = 'relu'),
        tf.keras.layers.Dense(16, activation = 'relu'),
        tf.keras.layers.Dense(1)
])

model.compile(optimizer = 'adam', loss = 'mean_squared_error')


for train_index, test_index in kf.split(X, Y):
    
    ## Splitting the data
    X_train , X_val = X.iloc[train_index], X.iloc[test_index]  
    Y_train, Y_val = Y.iloc[train_index], Y.iloc[test_index]    
    
    print("X_train shape is :", X_train.shape, "X_val shape is", X_val.shape)
    y_pred_list = []
    
    X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
    X_val = pd.DataFrame(scaler.fit_transform(X_val), columns = X_val.columns)

    model.fit(X_train, Y_train, verbose = 1, epochs = 20, batch_size = 32, validation_data = (X_val, Y_val))
    result = model.predict(X_val)
    
    result = pd.DataFrame(result)
    result.iloc[:, 0] = [0 if i <= 0 else i for i in result.iloc[:,0]]
    
    score = np.sqrt(mean_squared_error(Y_val, result))
    print('Fold ', str(fold), ' result is:', score, '\n')
    score_list_tf.append(score)

    test_preds_tf.append(model.predict(test))
    fold += 1

t2 = time.time()
print('TF model with cross validation take : {:.3f} sn.'.format(t2-t1))

X_train shape is : (52596, 12) X_val shape is (17532, 12)
Epoch 1/20
1644/1644 [==============================] - 3s 2ms/step - loss: 1.6558 - val_loss: 0.1152
Epoch 2/20
1644/1644 [==============================] - 2s 1ms/step - loss: 0.0896 - val_loss: 0.0696
Epoch 3/20
1644/1644 [==============================] - 3s 2ms/step - loss: 0.0650 - val_loss: 0.0597
Epoch 4/20
1644/1644 [==============================] - 2s 1ms/step - loss: 0.0595 - val_loss: 0.0569
Epoch 5/20
1644/1644 [==============================] - 2s 1ms/step - loss: 0.0556 - val_loss: 0.0503
Epoch 6/20
1644/1644 [==============================] - 2s 1ms/step - loss: 0.0511 - val_loss: 0.0519
Epoch 7/20
1644/1644 [==============================] - 2s 1ms/step - loss: 0.0481 - val_loss: 0.0457
Epoch 8/20
1644/1644 [==============================] - 2s 1ms/step - loss: 0.0472 - val_loss: 0.0446
Epoch 9/20
1644/1644 [==============================] - 3s 2ms/step - loss: 0.0462 - val_loss: 0.0443
Epoch 10/20
1644/1644 [=

In [23]:
mean = sum(score_list_tf) / len(score_list_tf)
variance = sum([((x - mean) ** 2) for x in score_list_tf]) / len(score_list_tf)
res = variance ** 0.5
print("Cross validation mean score:", sum(score_list_tf) / len(score_list_tf))
print("Cross validation score's Standart deviation is:", res)

Cross validation mean score: 0.20284938811241218
Cross validation score's Standart deviation is: 0.002370271378208629


In [24]:
test_preds_tf = pd.DataFrame(np.concatenate(test_preds_tf, axis = 1))
print(test_preds_tf.shape)

test_preds_tf = test_preds_tf.mean(axis = 1)
print(test_preds_tf.head())

(17520, 4)
0    6.176146
1    5.896741
2    5.810059
3    6.265101
4    5.308153
dtype: float32


In [25]:
submission['num_sold'] = np.exp(test_preds_tf)
submission.to_csv('TF_submission_4.csv', index = False)
submission.head()

,row_id,num_sold
0,70128,481.134094
1,70129,363.849854
2,70130,333.638672
3,70131,525.894958
4,70132,201.976852
